In [2]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
import pandas as pd

In [ ]:
# Pegar os locais definidos pela DC e geocodificar eles
# Pegar todas as coordenadas de latitude e longitude das ocorrencias unificadas
# Fazer um mapa de calor comparando os locais definidos pela DC e os locais observados pela analise do projeto

In [4]:
df = pd.read_csv(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Processados\BOMBEIROS_GEOCODED.csv", encoding="latin1")

In [9]:
geolocator = Nominatim(user_agent="geo_goiania")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def geocode_goiania(row):
    base_address = f"{row['ENDEREÇO']}, {row['BAIRRO']}, Goiânia, GO, Brasil"
    location = geocode(base_address)
    
    # Se falhar, tenta apenas bairro + Goiânia
    if location is None and pd.notna(row['BAIRRO']):
        location = geocode(f"{row['BAIRRO']}, Goiânia, GO, Brasil")
    
    # Se ainda falhar, tenta apenas Goiânia
    if location is None:
        location = geocode("Goiânia, GO, Brasil")
    
    # Retorna coordenadas
    if location:
        return pd.Series([location.latitude, location.longitude])
    else:
        return pd.Series([None, None])
    
LAT_MIN, LAT_MAX = -16.828, -16.546
LON_MIN, LON_MAX = -49.372, -49.043

def geocode_goiania(row):
    base_address = f"{row['ENDEREÇO']}, {row['BAIRRO']}, Goiânia, GO, Brasil"
    location = geocode(base_address)
    
    # fallback bairro + Goiânia
    if location is None and pd.notna(row['BAIRRO']):
        location = geocode(f"{row['BAIRRO']}, Goiânia, GO, Brasil")
    
    # fallback genérico Goiânia
    if location is None:
        location = geocode("Goiânia, GO, Brasil")
    
    # força coordenada dentro do bounding box
    if location:
        lat, lon = location.latitude, location.longitude
        if not (LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX):
            # ponto genérico central da cidade
            lat, lon = -16.6869, -49.2648
        return pd.Series([lat, lon])
    else:
        # fallback completo
        return pd.Series([-16.6869, -49.2648])

df[['LATITUDE', 'LONGITUDE']] = df.apply(geocode_goiania, axis=1)

NameError: name 'df' is not defined

In [ ]:
df = pd.read_csv("BOMBEIROS_GEOCODED.csv")


mapa = folium.Map(location=[-16.6869, -49.2648], zoom_start=12)  

for _, row in df.iterrows():
    if pd.notna(row['LATITUDE']) and pd.notna(row['LONGITUDE']):
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=f"{row['ENDEREÇO']} - {row['BAIRRO']}"
        ).add_to(mapa)


mapa.save("mapa_bombeiros.html")
